In [27]:
import pandas as pd
import workdays
import datetime
# from datetime import datetime, timedelta, date
from xbbg import blp
import pdblp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import AutoDateLocator

def F_BDP(T,Fund,F,F1,F2,F3):
   
    T1 = [t + ' JP Equity' for t in T]   
    BDP = blp.bdp(tickers=T1, flds=F).loc[T1,:]
    BDP.columns = F1 + F2 + F3
    BDP = BDP.reset_index(drop=False)
    BDP['index'] = Fund

    for j in F2:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in F3:
        BDP[j] = [round(x,2) for x in BDP[j].tolist()]
    
    return BDP
    

In [28]:
T = [
    "64315091",
    "6431417B",
    "64315239",
    "01312103",
    "0131416B",
    "01313103",
    "0131516B",
    "01314103"
]
Fund = ['世界IDX','グロコア','ロング10', '野村_保守', '野村_やや保守', '野村_普通', '野村_やや積極','野村_積極']
F = ['name_kanji','fund_incept_dt','fund_total_assets'] #'fund_comp_name'
F += ['volatility_360d','volatility_180d','current_trr_1yr','current_trr_6mo','current_trr_3mo','fund_mgr_stated_fee']

F1 = ['name','fund_incept_dt']
F2 = ['NAV']
F3 = ['vola_360d','vola_180d','trr_1yr','trr_6m','trr_3m','fee']

BDP2 = F_BDP(T,Fund,F,F1,F2,F3)
print(BDP2.shape)
BDP2

(8, 10)


,index,name,fund_incept_dt,NAV,vola_360d,vola_180d,trr_1yr,trr_6m,trr_3m,fee
0,世界IDX,世界経済ｲﾝﾃﾞｯｸｽﾌｧﾝﾄﾞ/SMTAM,2009-01-16,"380,242",10.45,11.93,14.63,4.96,2.18,0.55
1,グロコア,ｸﾞﾛｰﾊﾞﾙ経済ｺｱ/SMTAM,2017-11-30,"26,152",8.96,10.02,14.72,5.30,1.09,0.85
2,ロング10,ｸﾞﾛｰﾊﾞﾙ10資産ﾊﾞﾗﾝｽﾌｧﾝﾄﾞ,2023-09-29,"2,318",5.09,4.99,4.97,2.01,-0.36,0.60
3,野村_保守,のむﾗｯﾌﾟﾌｧﾝﾄﾞ保守型/野村,2010-03-15,"67,298",5.58,5.67,4.66,1.78,0.22,1.08
4,野村_やや保守,のむﾗｯﾌﾟ･ﾌｧﾝﾄﾞ(やや保守型),2016-11-11,"18,476",7.07,7.48,7.43,2.75,0.87,1.16
5,野村_普通,のむﾗｯﾌﾟﾌｧﾝﾄﾞ普通型/野村,2010-03-15,"528,686",8.57,9.21,10.42,4.00,1.49,1.23
6,野村_やや積極,のむﾗｯﾌﾟ･ﾌｧﾝﾄﾞ(やや積極型),2016-11-11,"64,045",11.09,12.27,15.47,6.05,2.80,1.30
7,野村_積極,のむﾗｯﾌﾟﾌｧﾝﾄﾞ積極型/野村,2010-03-15,"363,129",12.23,13.58,18.73,7.65,3.60,1.38


In [29]:
# T = ['03311187','0331418A','64315091','96311073','03312175','01313103','01314103']
# Fund = ['EM_SP','EM_オルカン','世界IDX','セゾン','EM_バランス','野村_普通','野村_積極']
T = ['0331418A','64315091','96311073','01313103','01314103','4731312A']
Fund = ['オルカン','世界IDX','セゾン','野村_普通','野村_積極','ソムリエ']
F = ['name_kanji','fund_incept_dt','fund_total_assets'] #'fund_comp_name'
F += ['volatility_360d','volatility_180d','current_trr_1yr','current_trr_6mo','current_trr_3mo','fund_mgr_stated_fee']

F1 = ['name','fund_incept_dt']
F2 = ['NAV']
F3 = ['vola_360d','vola_180d','trr_1yr','trr_6m','trr_3m','fee']

BDP = F_BDP(T,Fund,F,F1,F2,F3)
print(BDP.shape)
BDP

(6, 10)


,index,name,fund_incept_dt,NAV,vola_360d,vola_180d,trr_1yr,trr_6m,trr_3m,fee
0,オルカン,ｅＭＡＸＩＳ Ｓｌｉｍ 全世,2018-10-31,"5,526,155",14.85,16.89,28.03,11.82,6.27,0.10
1,世界IDX,世界経済ｲﾝﾃﾞｯｸｽﾌｧﾝﾄﾞ/SMTAM,2009-01-16,"380,242",10.45,11.93,14.63,4.96,2.18,0.55
2,セゾン,ｾｿﾞﾝ ｸﾞﾛｰﾊﾞﾙﾊﾞﾗﾝｽﾌｧﾝﾄﾞ,2007-03-15,"532,332",10.62,11.94,15.99,5.61,3.01,0.45
3,野村_普通,のむﾗｯﾌﾟﾌｧﾝﾄﾞ普通型/野村,2010-03-15,"528,686",8.57,9.21,10.42,4.00,1.49,1.23
4,野村_積極,のむﾗｯﾌﾟﾌｧﾝﾄﾞ積極型/野村,2010-03-15,"363,129",12.23,13.58,18.73,7.65,3.60,1.38
5,ソムリエ,投資のｿﾑﾘｴ/AMOne,2012-10-26,"315,537",3.97,4.12,1.90,0.81,-0.07,1.40


In [30]:
def BDH(T):
    T1 = [t + ' JP Equity' for t in T]  
    BDH = blp.bdh(T1, ['px_last','tot_return_index_gross_dvds'], d_from, d_to, Calendar='5D', Fill='P' ).reset_index()   
    
    df = pd.DataFrame(BDH['index'].values,columns = ['Date'])
    df['Date'] = pd.to_datetime(df['Date'])
    df = ME(df)
    tax = 0
    for i in range(len(T1)):
        df[Fund[i]+'_PR'] = BDH[T1[i]]['px_last'].pct_change()
        df[Fund[i]+'_GR'] = BDH[T1[i]]['tot_return_index_gross_dvds'].pct_change()
        df[Fund[i]+'_NR'] = (df[Fund[i]+'_GR'] - df[Fund[i]+'_PR'] ) * (1 - tax) + df[Fund[i]+'_PR']
        
    df = Adj(df)
        
    for i in range(len(T1)):
        for j in ['PR','GR','NR']:
            df[Fund[i]+'_'+j+'_Index'] = (1 + df[Fund[i]+'_'+j] ).cumprod().fillna(1) * 100

    return df


def Adj(df):
    tmp = df['世界IDX_PR'].first_valid_index()
    for j in ['PR','GR','NR']:
        df.loc[:tmp+5,'世界IDX_'+j] = df.loc[:tmp+5, 'セゾン_'+j].values
        
    tmp = ['MXWD Index','M2WD Index','USDJPY BGN Curncy','ES1 Index','TY1 Comdty','CO1 Comdty','GC1 Comdty']
    BDH1 = blp.bdh(tmp, ['px_last'], d_from, d_to, Calendar='5D', Fill='P' ).reset_index()
    BDH1.columns = ['Date'] + tmp
    BDH1['ACWI_PR'] = (BDH1['MXWD Index'] * BDH1['USDJPY BGN Curncy']).pct_change()
    BDH1['ACWI_GR'] = (BDH1['M2WD Index'] * BDH1['USDJPY BGN Curncy']).pct_change()
    BDH1['ACWI_NR'] = (BDH1['ACWI_GR'] - BDH1['ACWI_PR'] ) * (1 - 0.1) + BDH1['ACWI_GR']
    df = pd.concat([df, BDH1.iloc[:,1:]], axis=1)

    tmp = df['オルカン_PR'].first_valid_index()
    for j in ['PR','GR','NR']:
        df.loc[:tmp+5,'オルカン_'+j] = BDH1.loc[:tmp+5, 'ACWI_NR'].values
        
    tmp = ['USDJPY BGN Curncy','ES1 Index','TY1 Comdty','CO1 Comdty','GC1 Comdty']
    tmp1 = ['USDJPY','ES1','TY1','CO1','GC1']
    for j, j1 in zip(tmp, tmp1):
        df[j1+'_GR'] = BDH1[j].pct_change()
        df[j1+'_GR_Index'] = (1 + df[j1+'_GR'] ).cumprod().fillna(1) * 100

    # df['ES1_GR'] = df['ES1 Index'].pct_change()
    # df['ES1_GR_Index'] = (1 + df['ES1_GR'] ).cumprod().fillna(1) * 100
        
    return df

def ME(df):
    df['ME'] = 0
    tmp = df['Date'].dt.month != df['Date'].shift(-1).dt.month
    df['ME'] = np.where(tmp & (df['Date'].dt.month == 3), 3, np.where(tmp, 1, 0))
    df.loc[len(df)-1, 'ME'] = 3
    return df

In [31]:
d_from = '20070330'
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime('%Y%m%d')
df = BDH(T)

In [32]:
def Ret(k,L,df):
    L_index = [x + "_GR_Index" for x in L]
    tmp = df.loc[df["ME"]>=k,["Date"]+[x for x in L_index]].copy()
    tmp[L] = tmp[L_index].pct_change()
    tmp = tmp.iloc[1:].reset_index(drop=True)    
    return tmp[["Date"]+L]

def Percent(tmp):
    tmp.iloc[:, 1:] = tmp.iloc[:, 1:].applymap(lambda x: f"{x:.1%}")
    return tmp

def Corr(k,L,df):
    L_Index = [x + "_GR_Index" for x in L]
    tmp = df.loc[df["ME"]>=k,["Date"]+L_Index]
    tmp[L] =  tmp[L_Index].pct_change().iloc[1:]
    tmp = tmp[L].corr().applymap("{:.2f}".format)
        
    return tmp.replace("1.00", "-")

In [33]:
L = ['オルカン','セゾン','世界IDX','USDJPY','ES1','TY1','CO1','GC1']
Percent(Ret(3,L,df))

,Date,オルカン,セゾン,世界IDX,USDJPY,ES1,TY1,CO1,GC1
0,2008-03-31,-14.1%,-9.3%,-9.3%,-15.4%,-10.0%,12.4%,55.2%,31.3%
1,2009-03-31,-41.5%,-24.6%,-24.8%,-0.7%,-39.8%,10.0%,-65.2%,-1.7%
2,2010-03-31,51.3%,18.4%,27.8%,-5.5%,49.1%,-1.8%,40.0%,19.6%
3,2011-03-31,4.3%,-2.7%,-1.0%,-11.1%,15.2%,5.6%,37.8%,28.1%
4,2012-03-30,2.1%,1.5%,-0.8%,-0.3%,8.2%,11.7%,22.1%,15.3%
5,2013-03-29,29.7%,19.9%,20.6%,13.7%,13.3%,4.7%,0.0%,-5.3%
6,2014-03-31,31.4%,18.5%,14.4%,9.6%,21.1%,-2.7%,3.8%,-19.8%
7,2015-03-31,26.1%,17.2%,15.4%,16.4%,12.2%,6.5%,-53.0%,-8.1%
8,2016-03-31,-7.8%,-6.0%,-8.8%,-6.3%,1.3%,2.9%,-40.0%,4.1%
9,2017-03-31,17.2%,4.8%,6.5%,-1.0%,16.3%,-2.8%,19.3%,-0.3%


In [34]:
tmp = Corr(1,L,df)
print("Correlation")
tmp

Correlation


,オルカン,セゾン,世界IDX,USDJPY,ES1,TY1,CO1,GC1
オルカン,-,0.93,0.95,0.54,0.83,-0.36,0.48,-0.08
セゾン,0.93,-,0.97,0.60,0.71,-0.31,0.43,-0.06
世界IDX,0.95,0.97,-,0.55,0.73,-0.31,0.45,-0.02
USDJPY,0.54,0.60,0.55,-,0.07,-0.56,0.14,-0.39
ES1,0.83,0.71,0.73,0.07,-,-0.08,0.44,0.08
TY1,-0.36,-0.31,-0.31,-0.56,-0.08,-,-0.33,0.37
CO1,0.48,0.43,0.45,0.14,0.44,-0.33,-,0.11
GC1,-0.08,-0.06,-0.02,-0.39,0.08,0.37,0.11,-


In [35]:
def SD(k,L,Sort,df):
    
    L_Index = [x + "_GR_Index" for x in L]
    tmp = df.loc[df["ME"]>=k,["Date"]+L_Index].copy().reset_index(drop=True)
    # tmp = tmp.loc[r_from:,:].reset_index(drop=True)
    
    Year = (max(tmp["Date"]) -  min(tmp["Date"])).days/365.25

    Ret =  tmp.loc[len(tmp)-1,L_Index] / tmp.loc[0,L_Index]
    Ret =  Ret**(1/Year) - 1
    
    tmp[L] = tmp[L_Index].pct_change()
    tmp = tmp.iloc[1:].reset_index(drop=True)
    tmp = np.std(tmp[L]) * 12**0.5

    SD = pd.DataFrame({
        "Name": L,
        "Ret": Ret.values,
        "SD": tmp.values    })
    
    SD["Ratio"] = SD["Ret"] / SD["SD"]
    if Sort != "":
        SD = SD.sort_values(Sort, ascending=False).reset_index(drop=True)
    
    return SD


def Format(tmp):
    tmp["Ret"] = tmp["Ret"].apply("{:.2%}".format)
    tmp["SD"] = tmp["SD"].apply("{:.2%}".format)
    tmp["Ratio"] = tmp["Ratio"].apply("{:.2f}".format)
    return tmp

tmp = Format(SD(1,L,"Ratio",df))
print("SD",tmp.shape)
tmp

SD (8, 4)


,Name,Ret,SD,Ratio
0,オルカン,10.76%,18.86%,0.57
1,ES1,8.58%,15.81%,0.54
2,セゾン,5.66%,11.93%,0.47
3,世界IDX,5.36%,12.61%,0.43
4,GC1,6.10%,16.76%,0.36
5,TY1,1.74%,5.78%,0.30
6,USDJPY,1.59%,9.85%,0.16
7,CO1,0.86%,33.60%,0.03
